# 🚯 Lecture 11 Lab: Logistic regression and spam detection

<img src="https://github.com/joshuagrossman/mse125-labs-public/blob/main/hw5/img/spam-email.png?raw=1" alt= “spam-email” width="500" />

## ✅ Setup and data import
In this lab, we will work with a [classic dataset](https://archive.ics.uci.edu/dataset/94/spambase) of 4,601 emails classified as spam or not spam.

In [8]:
# Load in additional functions
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rcParams

# Use three digits past the decimal point and avoid scientific notation
pd.options.display.float_format = '{:.3f}'.format

# Format plots with a white background and dark features
sns.set_theme(style="whitegrid")

# Increase the default text size of plots
rcParams.update({'font.size': 20})

# Increase the default plot width and height
rcParams['figure.figsize'] = (12, 8)

# Read in the data
spam = pd.read_csv('https://jdgrossman.com/assets/spam.csv')

# Peek at 10 random rows
spam.head()

,make,address,all,3d,our,over,remove,internet,order,mail,...,char_semicolon,char_left_paren,char_left_bracket,char_exclamation,char_dollar,char_pound,capital_run_length_average,capital_run_length_longest,capital_run_length_total,is_spam
0,0.000,0.640,0.640,0.000,0.320,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.778,0.000,0.000,3.756,61,278,1
1,0.210,0.280,0.500,0.000,0.140,0.280,0.210,0.070,0.000,0.940,...,0.000,0.132,0.000,0.372,0.180,0.048,5.114,101,1028,1
2,0.060,0.000,0.710,0.000,1.230,0.190,0.190,0.120,0.640,0.250,...,0.010,0.143,0.000,0.276,0.184,0.010,9.821,485,2259,1
3,0.000,0.000,0.000,0.000,0.630,0.000,0.310,0.630,0.310,0.630,...,0.000,0.137,0.000,0.137,0.000,0.000,3.537,40,191,1
4,0.000,0.000,0.000,0.000,0.630,0.000,0.310,0.630,0.310,0.630,...,0.000,0.135,0.000,0.135,0.000,0.000,3.537,40,191,1


## ♨️ Warm up

How many emails are in the database?

What fraction of the emails in the database are spam?

Which email contains the highest percentage of words matching "money"? What percentage of words in that email match "money"?

In [12]:
# Your code here!

num_emails = len(spam)
print(num_emails)

spam_fraction = spam['is_spam'].mean()
print(spam_fraction)

idx = spam['money'].idxmax()
highest_money_email = spam.loc[idx]
highest_money_email['money'], idx

4601
0.39404477287546186


(np.float64(12.5), 545)

## 🎲 Linear probability models (LPMs)

Fit a linear regression model to the spam data with the `lm` function.

Use the following covariates to predict the likelihood that an email is spam:
- `char_dollar`
- `credit`
- `money`
- `re`

How would you interpret the model coefficients for the intercept and for `char_dollar`?

- Note: `char_dollar` represents the percentage of characters in the email that match `$`.

In [14]:
# Your code here!
import statsmodels.api as sm

X = spam[['char_dollar', 'credit', 'money', 're']]
X = sm.add_constant(X)
y = spam['is_spam']

lpm = sm.OLS(y, X).fit()
print(lpm.summary())


                            OLS Regression Results                            
Dep. Variable:                is_spam   R-squared:                       0.179
Model:                            OLS   Adj. R-squared:                  0.178
Method:                 Least Squares   F-statistic:                     250.3
Date:                Tue, 18 Nov 2025   Prob (F-statistic):          8.51e-195
Time:                        14:14:29   Log-Likelihood:                -2780.3
No. Observations:                4601   AIC:                             5571.
Df Residuals:                    4596   BIC:                             5603.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.3346      0.007     45.696      

Using your linear probability model and the `predict` function, predict the in-sample probability that each email is spam.

What is the smallest predicted probability? The largest? Do you notice any issues with these predictions?

In [15]:
# Your code here!
pred_probs = lpm.predict(X)
pred_min = pred_probs.min()
pred_max = pred_probs.max()
print(pred_min, pred_max)


-0.8125403869560923 3.8494089964108236


## 🎰 Odds functions

Write two functions:
- A function to convert probabilities to odds.
- A function to convert odds to probabilities

Test your functions by making sure that 2:1 odds returns a 2/3 probability, and vice versa.

Finally, suppose my probability of winning is 60%. If I double my odds of winning, what is my new probability of winning?

In [19]:
# Your code here!
def prob_to_odds(p):
    return p / (1 - p)

def odds_to_prob(o):
    return o / (1 + o)

# Tests
test1 = odds_to_prob(2)
test2 = prob_to_odds(2/3)
print(test1, test2)


# New probability of winning if odds of winning is changed
p = 0.60
o = prob_to_odds(p)
new_o = 2 * o
new_p = odds_to_prob(new_o)
print(new_p)


0.6666666666666666 1.9999999999999998
0.75


## 🪙 Fitting a logistic regression model

We can fit a logistic regression model with the same covariates as above with the following code:

In [21]:
import statsmodels.formula.api as smf

logit_model = smf.glm(
    "is_spam ~ char_dollar + credit + money + re",
    data=spam,
    family=sm.families.Binomial()
).fit()

print(logit_model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                is_spam   No. Observations:                 4601
Model:                            GLM   Df Residuals:                     4596
Model Family:                Binomial   Df Model:                            4
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2213.9
Date:                Tue, 18 Nov 2025   Deviance:                       4427.8
Time:                        14:18:25   Pearson chi2:                 1.00e+10
No. Iterations:                     8   Pseudo R-squ. (CS):             0.3152
Covariance Type:            nonrobust                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -1.0666      0.043    -24.680      

Interpret the intercept and `money` coefficients for the logistic regression model three different ways:
1. On the log odds scale
2. On the odds scale (by exponentiating the coefficients)
3. On the probability scale (using either the odds functions you wrote, or the divide by 4 trick).

Tip: Use the `coef` function to extract coefficients from the model.

In [22]:
# Your code here!
logit_model.params


,0
Intercept,-1.067
char_dollar,11.818
credit,2.312
money,1.993
re,-0.776
